In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Impdar is a dependency here: download from https://github.com/dlilien/ImpDAR
from impdar.lib.ApresData.load_quadpol import load_quadpol_fujita
from impdar.lib.ApresData._QuadPolProcessing import power_anomaly, lowpass

%matplotlib notebook

from effmed.lib.matrix_model import effective_medium
from effmed.lib.supplemental import dB, fresnel

### Example 1: Uniform

In [ ]:
em = effective_medium()

fc = 300e6
em.system_setup(fc)

Temp = 253.
epsr = 3.15
em.epsr = epsr
chis = [.35,.15,0.5]
em.ice_properties(T=Temp,epsr=epsr,chi=chis)

H = 1000
zs = np.arange(10,H)
thetas=0.
psis = 0.
em.solve(zs,H,thetas,psis,chis)

dat1 = load_quadpol_fujita(em)
dat1.rotational_transform(n_thetas=201)
dat1.find_cpe()
dat1.coherence2d(.15,2.)

### Example 2: Strengthening Girdle

In [ ]:
em = effective_medium()

fc = 300e6
em.system_setup(fc)

Temp = 253.
epsr = 3.15
em.epsr = epsr
chis = [.26,.24,0.5]
em.ice_properties(T=Temp,epsr=epsr,chi=chis)

H = 1000
layer_dz = 2.1
zs = np.arange(.1,H+layer_dz,layer_dz)
thetas = np.zeros(len(zs))
psis = np.zeros(len(zs))
dzs = layer_dz*np.ones(len(zs))
chi1 = np.linspace(.26,.4,len(zs))
chi2 = np.linspace(.24,.1,len(zs))
chi3 = chis[2]*np.ones(len(zs))
chis = np.transpose([chi1,chi2,chi3])
em.solve(zs,dzs,thetas,psis,chis)

dat2 = load_quadpol_fujita(em)
dat2.rotational_transform(n_thetas=201)
dat2.find_cpe()
dat2.coherence2d(.15,layer_dz+.1)

### Example 3: Anisotropic Reflectivity (no birefringence)

In [ ]:
# Initialize the model at a chosen center frequency
em = effective_medium()

fc = 300e6
em.system_setup(fc)

Temp = 253.
epsr = 3.15
em.epsr = epsr
chis = [.333,.333,.333]
em.ice_properties(T=Temp,epsr=epsr,chi=chis)

H = 1000
zs = np.arange(10,H)
thetas=0.
psis = 0.001
gammas = np.array([1.,.01])
em.solve(zs,H,thetas,psis,chis,gammas=gammas)

dat3 = load_quadpol_fujita(em)
dat3.rotational_transform(n_thetas=201)
dat3.find_cpe()
dat3.coherence2d(.15,layer_dz+.1)

### Example 4: Anisotropic Reflectivity (with birefringence)

In [ ]:
# Initialize the model at a chosen center frequency
em = effective_medium()

fc = 300e6
em.system_setup(fc)

Temp = 253.
epsr = 3.15
em.epsr = epsr
chis = [.35,.15,0.5]
em.ice_properties(T=Temp,epsr=epsr,chi=chis)

H = 1000
zs = np.arange(10,H)
thetas=0.
psis = 0.
gammas = np.array([1.,.2])
em.solve(zs,H,thetas,psis,chis,gammas=gammas)

dat4 = load_quadpol_fujita(em)
dat4.rotational_transform(n_thetas=201)
dat4.find_cpe()
dat4.coherence2d(.15,2.)

### Example 5: Instantaneous Rotation

In [ ]:
em = effective_medium()

fc = 300e6
em.system_setup(fc)

Temp = 253.
epsr = 3.15
em.epsr = epsr
chis = [.35,.15,0.5]
em.ice_properties(T=Temp,epsr=epsr,chi=chis)

H = 1000
layer_dz = 2.
zs = np.arange(1,H,layer_dz)
thetas = np.zeros(len(zs))
psis5 = np.zeros_like(zs) #.3*np.tanh(.01*(zs-400))+.3
psis5[zs>300] = .1*np.pi
dzs = layer_dz*np.ones(len(zs))
chis = np.tile(chis,(len(zs),1))
em.solve(zs,dzs,thetas,psis5,chis)

dat5 = load_quadpol_fujita(em)
dat5.rotational_transform(n_thetas=201)
dat5.find_cpe(Wn=3e7,rad_start=0.1*np.pi,rad_end=.6*np.pi)
dat5.coherence2d(.15,layer_dz+.1)

### Example 6: Continuous Rotation

In [ ]:
em = effective_medium()

fc = 300e6
em.system_setup(fc)

Temp = 253.
epsr = 3.15
em.epsr = epsr
chis = [.35,.15,0.5]
em.ice_properties(T=Temp,epsr=epsr,chi=chis)

H = 1000
layer_dz = 2.
zs = np.arange(1,H,layer_dz)
thetas = np.zeros(len(zs))
psis6 = .3*np.tanh(.01*(zs-400))+.3
dzs = layer_dz*np.ones(len(zs))
chis = np.tile(chis,(len(zs),1))
em.solve(zs,dzs,thetas,psis6,chis)

dat6 = load_quadpol_fujita(em)
dat6.rotational_transform(n_thetas=201)
dat6.find_cpe(Wn=3e7)
dat6.coherence2d(.15,layer_dz+.1)

### Plot all examples together

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(8,7))

dats = [dat1,dat2,dat3,dat4,dat5,dat6]
labels = ['A','B','C','D','E','F','G','H','I','J','K','L']

for i,dat in enumerate(dats):
    Θs,Ds = np.meshgrid(dat.thetas,dat.range)

    ax1 = plt.subplot(3,4,i*2+1,facecolor='k')
    plt.tick_params(labelleft=False)
    im1 = plt.pcolormesh(Θs,Ds,np.real(dB(dat.HV**2.)),cmap='Greys_r',vmin=-10,vmax=0.)
    ax1.text(0.09,.92,labels[i*2],transform=ax1.transAxes,fontweight='bold',fontsize=10,ha='center',va='center',
           bbox=dict(facecolor='w', edgecolor='k', pad=2.0, linewidth=1.))
    plt.plot(dat.cpe,Ds,'m.',ms=.5)
    if i == 4:
        plt.plot(psis5+np.pi/2.,Ds,'.',c='lime',ms=.5)
    elif i == 5:
        plt.plot(psis6+np.pi/2.,Ds,'.',c='lime',ms=.5)
    plt.ylim(1000,0)
    plt.xticks([0,np.pi/2.,np.pi])
    ax1.set_xticklabels(['','',''])

    ax2 = plt.subplot(3,4,i*2+2)
    plt.tick_params(labelleft=False)
    im2 = plt.pcolormesh(Θs,Ds,np.angle(dat.chhvv),cmap='twilight_shifted',vmin=-np.pi,vmax=np.pi)
    ax2.text(0.09,.92,labels[i*2+1],transform=ax2.transAxes,fontweight='bold',fontsize=10,ha='center',va='center',
           bbox=dict(facecolor='w', edgecolor='k', pad=2.0, linewidth=1.))
    plt.ylim(1000,0)
    plt.xticks([0,np.pi/2.,np.pi])
    ax2.set_xticklabels(['','',''])

# -----------------------------------------------------------------------------------
# Colorbars
# -----------------------------------------------------------------------------------

fig.subplots_adjust(right=0.87)
cbar1_ax = fig.add_axes([0.9, 0.55, 0.01, 0.3])
cbar2_ax = fig.add_axes([0.9, 0.15, 0.01, 0.3])
fig.colorbar(im1,cax=cbar1_ax,label='Cross-Polarized Power (dB)',extend='min',orientation='vertical',ticks=[-10,-5,0])
cbar2 = fig.colorbar(im2,cax=cbar2_ax,label='Phase Angle ($\phi_{HHVV}$)',orientation='vertical',ticks=[-np.pi,0,np.pi])
cbar2.ax.set_yticklabels(['-π','0','π'])

plt.savefig('eff_med_suite.png',dpi=300)

### Assign reflectivity with fabric using Fresnel Equations

In [ ]:
em = effective_medium()

fc = 300e6
em.system_setup(fc)

Temp = 253.
epsr = 3.15
em.epsr = epsr
chis = [.34,.33,.33]
em.ice_properties(T=Temp,epsr=epsr,chi=chis)

H = 1000
layer_dz = 2.1
zs = np.arange(.1,H+layer_dz,layer_dz)
thetas = np.zeros(len(zs))
psis = 0.001 + np.zeros(len(zs))
dzs = layer_dz*np.ones(len(zs))
chi1 = .375+(1.+np.tanh((zs-400)*.005))/6.
chi2 = np.linspace(.25,.15,len(zs))
chi3 = 1. - chi1 - chi2
chis = np.transpose([chi1,chi2,chi3])
gammas = np.transpose([np.ones(len(zs)),fresnel(chis)])
em.solve(zs,dzs,thetas,psis,chis,gammas=gammas)

dat = load_quadpol_fujita(em)
dat.rotational_transform(n_thetas=201)
dat.find_cpe(Wn=3e7)
dat.coherence2d(.016,5.)

In [ ]:
Θs,Ds = np.meshgrid(dat.thetas,dat.range)

plt.figure(figsize=(6,6))

ax1 = plt.subplot(231)
plt.tick_params()
plt.pcolormesh(Θs,Ds/1000.,np.real(dB(dat.HH**2.)),cmap='Greys_r',vmin=-10,vmax=0.)
ax1.text(0.1,.9,'A',transform=ax1.transAxes,fontweight='bold',fontsize=12,ha='center',va='center',
       bbox=dict(facecolor='w', edgecolor='k', pad=2.0, linewidth=1.))
plt.ylim(1,0)
plt.xticks([0,np.pi/2.,np.pi])
ax1.set_xticklabels(['0','π/2','π'])
plt.xlabel('Azimuth ($\psi$)')
plt.ylabel('Depth (km)')
cbar = plt.colorbar(label='Cross-Pol. Power (dB)',extend='min',location='top',ticks=[-10,-5,0])

ax2 = plt.subplot(232,facecolor='k')
plt.tick_params(labelleft=False)
plt.pcolormesh(Θs,Ds,np.real(dB(dat.HV**2.)),cmap='Greys_r',vmin=-10,vmax=0.)
ax2.text(0.1,.9,'B',transform=ax2.transAxes,fontweight='bold',fontsize=12,ha='center',va='center',
       bbox=dict(facecolor='w', edgecolor='k', pad=2.0, linewidth=1.))
plt.plot(dat.cpe,Ds,'m.',ms=.5)
plt.ylim(1000,0)
plt.xticks([0,np.pi/2.,np.pi])
ax2.set_xticklabels(['','',''])
#plt.xlabel('Azimuth ($\psi$)')
cbar = plt.colorbar(label='Co-Pol. Power (dB)',extend='min',location='top',ticks=[-10,-5,0])

ax3 = plt.subplot(233)
plt.tick_params(labelleft=False)
plt.pcolormesh(Θs,Ds,np.angle(dat.chhvv),cmap='twilight_shifted',vmin=-np.pi,vmax=np.pi)
ax3.text(0.1,.9,'C',transform=ax3.transAxes,fontweight='bold',fontsize=12,ha='center',va='center',
       bbox=dict(facecolor='w', edgecolor='k', pad=2.0, linewidth=1.))
plt.ylim(1000,0)
plt.xticks([0,np.pi/2.,np.pi])
ax3.set_xticklabels(['','',''])
#plt.xlabel('Azimuth ($\psi$)')
cbar = plt.colorbar(label='Phase Angle ($\phi_{HHVV}$)',location='top',ticks=[-np.pi,0,np.pi])
cbar.ax.set_xticklabels(['-π','0','π'])

ax4 = plt.subplot(235)
plt.tick_params(labelleft=False)
plt.plot(chi1,dat.range,'k',label='$\lambda_1$')
plt.plot(chi2,dat.range,'grey',label='$\lambda_2$')
plt.plot(chi3,dat.range,'lightgrey',label='$\lambda_3$')
ax4.text(0.1,.9,'E',transform=ax4.transAxes,fontweight='bold',fontsize=12,ha='center',va='center',
       bbox=dict(facecolor='w', edgecolor='k', pad=2.0, linewidth=1.))
plt.ylim(1000,0)
plt.legend()
plt.xlabel('Eigenvalue')

ax5 = plt.subplot(234)
plt.tick_params()
plt.semilogx(gammas[:,1]/gammas[:,0],dat.range/1000.,'k')
ax5.text(0.1,.9,'D',transform=ax5.transAxes,fontweight='bold',fontsize=12,ha='center',va='center',
       bbox=dict(facecolor='w', edgecolor='k', pad=2.0, linewidth=1.))
plt.ylim(1,0)
plt.axvline(1,color='k',ls='--')
plt.xlabel('Reflection Ratio')
plt.ylabel('Depth (km)')

plt.tight_layout()
plt.savefig('./Reflection_Ratio_Polarimetry.png',dpi=300)